# Parameter Management

Once we have chosen an architecture
and set our hyperparameters,
we proceed to the training loop,
where our goal is to find parameter values
that minimize our loss function.
After training, we will need these parameters
in order to make future predictions.
Additionally, we will sometimes wish
to extract the parameters
perhaps to reuse them in some other context,
to save our model to disk so that
it may be executed in other software,
or for examination in the hope of
gaining scientific understanding.

Most of the time, we will be able
to ignore the nitty-gritty details
of how parameters are declared
and manipulated, relying on deep learning frameworks
to do the heavy lifting.
However, when we move away from
stacked architectures with standard layers,
we will sometimes need to get into the weeds
of declaring and manipulating parameters.
In this section, we cover the following:

* Accessing parameters for debugging, diagnostics, and visualizations.
* Sharing parameters across different model components.


In [1]:
import torch
from torch import nn

(**We start by focusing on an MLP with one hidden layer.**)


In [7]:
net = nn.Sequential(nn.LazyLinear(8),
                    nn.ReLU(),
                    nn.LazyLinear(1))

X = torch.rand(size=(2, 4))
net(X).shape

torch.Size([2, 1])

In [8]:
type(net(X)), type(net), net.__class__.__mro__

(torch.Tensor,
 torch.nn.modules.container.Sequential,
 (torch.nn.modules.container.Sequential,
  torch.nn.modules.module.Module,
  object))

## [**Parameter Access**]
:label:`subsec_param-access`

Let's start with how to access parameters
from the models that you already know.


When a model is defined via the `Sequential` class,
we can first access any layer by indexing
into the model as though it were a list.
Each layer's parameters are conveniently
located in its attribute.


We can inspect the parameters of the second fully connected layer as follows.


In [10]:
net[2].state_dict()

OrderedDict([('weight',
              tensor([[ 0.1304, -0.1833, -0.3206, -0.1466,  0.1805, -0.1754, -0.0566,  0.2474]])),
             ('bias', tensor([0.0943]))])

In [18]:
X, net[1].state_dict()

(tensor([[0.0292, 0.8661, 0.3945, 0.1812],
         [0.3214, 0.5731, 0.0150, 0.1906]]),
 OrderedDict())

We can see that this fully connected layer
contains two parameters,
corresponding to that layer's
weights and biases, respectively.


### [**Targeted Parameters**]

Note that each parameter is represented
as an instance of the parameter class.
To do anything useful with the parameters,
we first need to access the underlying numerical values.
There are several ways to do this.
Some are simpler while others are more general.
The following code extracts the bias
from the second neural network layer, which returns a parameter class instance, and
further accesses that parameter's value.


In [20]:
type(net[2].bias), net[2].bias.data

(torch.nn.parameter.Parameter, tensor([0.0943]))

Parameters are complex objects,
containing values, gradients,
and additional information.
That is why we need to request the value explicitly.

In addition to the value, each parameter also allows us to access the gradient. Because we have not invoked backpropagation for this network yet, it is in its initial state.


In [26]:
net[2].weight.grad == None

True

### [**All Parameters at Once**]

When we need to perform operations on all parameters,
accessing them one-by-one can grow tedious.
The situation can grow especially unwieldy
when we work with more complex, e.g., nested, modules,
since we would need to recurse
through the entire tree to extract
each sub-module's parameters. Below we demonstrate accessing the parameters of all layers.


In [32]:
list( net.named_modules() )

[('',
  Sequential(
    (0): Linear(in_features=4, out_features=8, bias=True)
    (1): ReLU()
    (2): Linear(in_features=8, out_features=1, bias=True)
  )),
 ('0', Linear(in_features=4, out_features=8, bias=True)),
 ('1', ReLU()),
 ('2', Linear(in_features=8, out_features=1, bias=True))]

In [33]:
list(net.named_parameters())

[('0.weight',
  Parameter containing:
  tensor([[-0.0633, -0.3307,  0.0600, -0.2094],
          [-0.1911, -0.3083, -0.2251,  0.0730],
          [-0.3256,  0.4791, -0.0506,  0.2389],
          [ 0.0836, -0.4369,  0.1251,  0.1017],
          [-0.0584,  0.1450, -0.1022, -0.3566],
          [ 0.2195, -0.4467, -0.3222, -0.1818],
          [-0.4417, -0.3197, -0.4090, -0.3243],
          [ 0.0065, -0.1843,  0.0570,  0.3704]], requires_grad=True)),
 ('0.bias',
  Parameter containing:
  tensor([-0.4802, -0.3730, -0.3646, -0.2636, -0.3703, -0.1585,  0.1012, -0.1698],
         requires_grad=True)),
 ('2.weight',
  Parameter containing:
  tensor([[ 0.1304, -0.1833, -0.3206, -0.1466,  0.1805, -0.1754, -0.0566,  0.2474]],
         requires_grad=True)),
 ('2.bias',
  Parameter containing:
  tensor([0.0943], requires_grad=True))]

In [28]:
[(name, param.shape) for name, param in net.named_parameters()]

[('0.weight', torch.Size([8, 4])),
 ('0.bias', torch.Size([8])),
 ('2.weight', torch.Size([1, 8])),
 ('2.bias', torch.Size([1]))]

## [**Tied Parameters**]

Often, we want to share parameters across multiple layers.
Let's see how to do this elegantly.
In the following we allocate a fully connected layer
and then use its parameters specifically
to set those of another layer.
Here we need to run the forward propagation
`net(X)` before accessing the parameters.


In [41]:
shared = nn.LazyLinear(8)
net = nn.Sequential(nn.LazyLinear(8), nn.ReLU(),
                    shared, nn.ReLU(),
                    shared, nn.ReLU(),
                    nn.LazyLinear(1))

net[2].state_dict()

OrderedDict([('weight', <UninitializedParameter>),
             ('bias', <UninitializedParameter>)])

In [43]:
# We need to give the shared layer a name so that we can refer to its
# parameters
shared = nn.LazyLinear(8)
net = nn.Sequential(nn.LazyLinear(8), nn.ReLU(),
                    shared, nn.ReLU(),
                    shared, nn.ReLU(),
                    nn.LazyLinear(1))

net(X)
# Check whether the parameters are the same
print(net[2].weight.data[0] == net[4].weight.data[0])
net[2].weight.data[0, 0] = 100
# Make sure that they are actually the same object rather than just having the
# same value
print(net[2].weight.data[0] == net[4].weight.data[0])
print(net[2].state_dict())

tensor([True, True, True, True, True, True, True, True])
tensor([True, True, True, True, True, True, True, True])
OrderedDict([('weight', tensor([[ 1.0000e+02, -1.2555e-01,  5.7013e-02, -3.1229e-01, -9.8061e-02,
         -9.6283e-02, -8.3007e-02,  8.7215e-02],
        [-2.5946e-01,  5.6948e-02, -2.5267e-03,  1.4576e-01,  3.3833e-01,
          2.2483e-01, -4.9014e-02, -2.1215e-01],
        [ 7.3850e-02, -3.6631e-03, -3.2373e-01,  1.0161e-01, -1.0198e-01,
          3.0227e-01, -2.1811e-01, -3.3762e-01],
        [ 1.9422e-02, -6.9333e-02, -3.0862e-01,  1.3122e-01, -3.3249e-02,
         -3.0657e-01, -2.8865e-01,  3.3427e-01],
        [ 2.0247e-01, -1.6475e-02, -1.8619e-01, -1.0867e-01, -7.2850e-02,
          1.3016e-01,  3.3604e-01, -8.5366e-02],
        [-2.9673e-02, -2.6704e-01,  3.5169e-01, -2.7723e-01,  2.5134e-01,
          1.5577e-01, -5.1612e-02, -1.6731e-02],
        [-1.2054e-02, -2.0473e-01, -2.8386e-02,  9.0748e-02,  1.6592e-01,
         -1.3218e-01, -1.6135e-01, -2.4300e-01],
 

In [ ]:
# We need to give the shared layer a name so that we can refer to its
# parameters


OrderedDict([('weight', <UninitializedParameter>),
             ('bias', <UninitializedParameter>)])

This example shows that the parameters
of the second and third layer are tied.
They are not just equal, they are
represented by the same exact tensor.
Thus, if we change one of the parameters,
the other one changes, too.


You might wonder,
when parameters are tied
what happens to the gradients?
Since the model parameters contain gradients,
the gradients of the second hidden layer
and the third hidden layer are added together
during backpropagation.


## Summary

We have several ways of accessing and tying model parameters.


## Exercises

1. Use the `NestMLP` model defined in :numref:`sec_model_construction` and access the parameters of the various layers.
1. Construct an MLP containing a shared parameter layer and train it. During the training process, observe the model parameters and gradients of each layer.
1. Why is sharing parameters a good idea?


[Discussions](https://discuss.d2l.ai/t/57)
